In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('Basic_Stats.csv')
print(df.shape)
df['Tournament App?'] = df['School'].str.contains('NCAA', case=True)*1 # 1 if tournament appearance
df.head()

(9802, 33)


,School,Games Played,Wins,Losses,W-L%,Simple Rating System,Strength of Schedule,Conf. Wins,Conf. Losses,Home Wins,Home Losses,Away Wins,Away Losses,Total Pts Scored,Total Pts Suffered,Minutes Played,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Tournament App?
0,Air Force,28,5,23,0.179,-5.62,5.05,1.0,17.0,5.0,10.0,0.0,10.0,1880.0,2089,NaN,690.0,1684.0,0.410,130.0,414.0,0.314,370.0,589.0,0.628,NaN,1040.0,385.0,154.0,79.0,403.0,555.0,1996,0
1,Akron,26,3,23,0.115,-11.16,2.52,0.0,18.0,2.0,11.0,1.0,12.0,1631.0,1949,NaN,575.0,1409.0,0.408,114.0,361.0,0.316,367.0,528.0,0.695,NaN,782.0,307.0,147.0,60.0,372.0,454.0,1996,0
2,Alabama,32,19,13,0.594,6.99,6.11,9.0,7.0,12.0,2.0,7.0,6.0,2303.0,2275,NaN,843.0,1974.0,0.427,255.0,700.0,0.364,362.0,546.0,0.663,NaN,1252.0,408.0,197.0,196.0,440.0,567.0,1996,0
3,Alabama State,27,9,18,0.333,-23.42,-12.79,5.0,9.0,5.0,8.0,4.0,6.0,1995.0,2226,NaN,713.0,1677.0,0.425,178.0,487.0,0.366,391.0,607.0,0.644,NaN,834.0,289.0,225.0,51.0,326.0,482.0,1996,0
4,Alcorn State,25,10,15,0.400,-18.20,-9.20,7.0,7.0,7.0,4.0,3.0,10.0,1942.0,2088,1005.0,721.0,1623.0,0.444,121.0,403.0,0.300,379.0,576.0,0.658,NaN,1009.0,354.0,233.0,90.0,472.0,558.0,1996,0


In [13]:
# Introduce per game stats
for per_game_col in df.columns[13:32]:
    if '%' in per_game_col:
        continue
    else:
        df[per_game_col+' Per Game'] = df[per_game_col]/df['Games Played']
df.head()

,School,Games Played,Wins,Losses,W-L%,Simple Rating System,Strength of Schedule,Conf. Wins,Conf. Losses,Home Wins,Home Losses,Away Wins,Away Losses,Total Pts Scored,Total Pts Suffered,Minutes Played,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Year,Tournament App?,Total Pts Scored Per Game,Total Pts Suffered Per Game,Minutes Played Per Game,FGM Per Game,FGA Per Game,3PM Per Game,3PA Per Game,FTM Per Game,FTA Per Game,ORB Per Game,TRB Per Game,AST Per Game,STL Per Game,BLK Per Game,TOV Per Game,PF Per Game
0,Air Force,28,5,23,0.179,-5.62,5.05,1.0,17.0,5.0,10.0,0.0,10.0,1880.0,2089,NaN,690.0,1684.0,0.410,130.0,414.0,0.314,370.0,589.0,0.628,NaN,1040.0,385.0,154.0,79.0,403.0,555.0,1996,0,67.142857,74.607143,NaN,24.642857,60.142857,4.642857,14.785714,13.214286,21.035714,NaN,37.142857,13.750000,5.500000,2.821429,14.392857,19.821429
1,Akron,26,3,23,0.115,-11.16,2.52,0.0,18.0,2.0,11.0,1.0,12.0,1631.0,1949,NaN,575.0,1409.0,0.408,114.0,361.0,0.316,367.0,528.0,0.695,NaN,782.0,307.0,147.0,60.0,372.0,454.0,1996,0,62.730769,74.961538,NaN,22.115385,54.192308,4.384615,13.884615,14.115385,20.307692,NaN,30.076923,11.807692,5.653846,2.307692,14.307692,17.461538
2,Alabama,32,19,13,0.594,6.99,6.11,9.0,7.0,12.0,2.0,7.0,6.0,2303.0,2275,NaN,843.0,1974.0,0.427,255.0,700.0,0.364,362.0,546.0,0.663,NaN,1252.0,408.0,197.0,196.0,440.0,567.0,1996,0,71.968750,71.093750,NaN,26.343750,61.687500,7.968750,21.875000,11.312500,17.062500,NaN,39.125000,12.750000,6.156250,6.125000,13.750000,17.718750
3,Alabama State,27,9,18,0.333,-23.42,-12.79,5.0,9.0,5.0,8.0,4.0,6.0,1995.0,2226,NaN,713.0,1677.0,0.425,178.0,487.0,0.366,391.0,607.0,0.644,NaN,834.0,289.0,225.0,51.0,326.0,482.0,1996,0,73.888889,82.444444,NaN,26.407407,62.111111,6.592593,18.037037,14.481481,22.481481,NaN,30.888889,10.703704,8.333333,1.888889,12.074074,17.851852
4,Alcorn State,25,10,15,0.400,-18.20,-9.20,7.0,7.0,7.0,4.0,3.0,10.0,1942.0,2088,1005.0,721.0,1623.0,0.444,121.0,403.0,0.300,379.0,576.0,0.658,NaN,1009.0,354.0,233.0,90.0,472.0,558.0,1996,0,77.680000,83.520000,40.2,28.840000,64.920000,4.840000,16.120000,15.160000,23.040000,NaN,40.360000,14.160000,9.320000,3.600000,18.880000,22.320000


In [14]:
# Let's focus the analysis to 2019-2010 to not have to bother with NaN entries for now
df = df[(df['Year'] >= 2010) & (df['Year'] <= 2019)]
df.columns

Index(['School', 'Games Played', 'Wins', 'Losses', 'W-L%',
       'Simple Rating System', 'Strength of Schedule', 'Conf. Wins',
       'Conf. Losses', 'Home Wins', 'Home Losses', 'Away Wins', 'Away Losses',
       'Total Pts Scored', 'Total Pts Suffered', 'Minutes Played', 'FGM',
       'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'Year', 'Tournament App?',
       'Total Pts Scored Per Game', 'Total Pts Suffered Per Game',
       'Minutes Played Per Game', 'FGM Per Game', 'FGA Per Game',
       '3PM Per Game', '3PA Per Game', 'FTM Per Game', 'FTA Per Game',
       'ORB Per Game', 'TRB Per Game', 'AST Per Game', 'STL Per Game',
       'BLK Per Game', 'TOV Per Game', 'PF Per Game'],
      dtype='object')

In [15]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

df = df.dropna()

X_train, X_test, y_train, y_test = train_test_split(df[['Minutes Played Per Game', 'FGM Per Game', 'FGA Per Game',
       '3PM Per Game', '3PA Per Game', 'FTM Per Game', 'FTA Per Game',
       'ORB Per Game', 'TRB Per Game', 'AST Per Game', 'STL Per Game',
       'BLK Per Game', 'TOV Per Game', 'PF Per Game']], df['Tournament App?'], test_size=0.2, random_state=40)

model = LogisticRegression(max_iter=1000)
rfe = RFE(model, n_features_to_select=3)
rfe = rfe.fit(X_train, y_train)
model.fit(X_train, y_train)

print("Logist Reg w/ All Pred: {:.2f}%".format(100*model.score(X_test, y_test)))
print("Logist Reg w/ All Pred Conf Mat: {0}".format(confusion_matrix(y_test, model.predict(X_test))))
X_test_rfemod = X_test.iloc[:, list(np.where(rfe.ranking_ == 1)[0])]
print("Logist Reg w/ 3 Most Relevant Pred: {:.2f}%".format(100*rfe.estimator_.score(X_test_rfemod, y_test)))
print("Logist Reg w/ 3 Most Relevant Pred Conf Mat: {0}".format(confusion_matrix(y_test, rfe.estimator_.predict(X_test_rfemod))))

Logist Reg w/ All Pred: 86.00%
Logist Reg w/ All Pred Conf Mat: [[545  20]
 [ 77  51]]
Logist Reg w/ 3 Most Relevant Pred: 83.41%
Logist Reg w/ 3 Most Relevant Pred Conf Mat: [[548  17]
 [ 98  30]]


c:\Users\pghme\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\pghme\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [16]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso = Lasso(alpha=0.01, max_iter=1000) # This only works with CV
lasso.fit(X_train_scaled, y_train)

# selected_features = np.where(lasso.coef_ != 0)[0]

print("Lasso Reg: {:.2f}%".format(100*lasso.score(X_test_scaled, y_test)))

Lasso Reg: 27.57%


In [20]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=40)
rf.fit(X_train, y_train)
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_features = indices[:3]

print("Random Forest w/ All Pred: {:.2f}%".format(100*rf.score(X_test, y_test)))
rf_top3 = RandomForestClassifier(n_estimators=100, random_state=40)
rf_top3.fit(X_train.iloc[:, top_features], y_train)
print("Random Forest w/ 3 Most Relevant Pred: {:.2f}%".format(100*rf_top3.score(X_test.iloc[:, top_features], y_test)))
print(X_train.columns[top_features[0]], X_train.columns[top_features[1]], X_train.columns[top_features[2]])


Random Forest w/ All Pred: 85.43%
Random Forest w/ 3 Most Relevant Pred: 78.07%
TOV Per Game AST Per Game PF Per Game
